In [ ]:
# Import library
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder #OneHotEncoder #StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import average_precision_score
from sklearn.metrics import ndcg_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import numpy as np
import joblib
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Load dataset
file_path = 'C:/Users/User/OneDrive/Documents/KULIAH/SEMESTER 7/SKRIPSI/WEB FORUM DISKUSI GAME/AI/DATA SET/WEB REKOMENDASI HERO 1/DI NUMERIK DI JKT/STREAMLIT/WEB REKOMENDASI HERO/hero_data.csv'
df = pd.read_csv(file_path)

# Tampilkan data awal
print("Dataset Awal:")
print(df.head())

Dataset Awal:
  hero_name         hero_role        hero_specially  \
0   Alucard  Fighter,Assassin  Charge,Crowd Control   
1     Bruno          Marksman                  Reap   
2     Fanny          Assassin           Charge,Reap   
3  Lancelot          Assassin   Charge,Burst Damage   
4    Claude          Marksman            Reap,Burst   

   hero_overview.hero_durability  hero_overview.hero_offence  \
0                             45                          87   
1                             42                          90   
2                             60                          85   
3                             50                          96   
4                             31                          88   

   hero_overview.hero_ability  hero_overview.hero_difficulty  
0                          50                             45  
1                          68                             46  
2                          50                            100  
3                 

In [11]:
# --- Data Cleaning ---
# Hapus baris yang memiliki nilai 0 pada atribut numerik
df = df[
    (df['hero_overview.hero_durability'] != 0) &
    (df['hero_overview.hero_offence'] != 0) &
    (df['hero_overview.hero_ability'] != 0) &
    (df['hero_overview.hero_difficulty'] != 0)
]

# Tampilkan jumlah data setelah dibersihkan
print("Jumlah data setelah data cleaning:", df.shape[0])


Jumlah data setelah data cleaning: 67


In [12]:
# Encode kolom kategorikal
categorical_columns = ['hero_name','hero_role', 'hero_specially']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

    # Tampilkan mapping asli ke encoded
    print(f"\nMapping {col} (Asli -> Encoded):")
    for i, value in enumerate(le.classes_):
        print(f"{value}: {i}")

joblib.dump(label_encoders, 'label_encoders.pkl')


Mapping hero_name (Asli -> Encoded):
Akai: 0
Aldous: 1
Alice: 2
Alpha: 3
Alucard: 4
Angela: 5
Argus: 6
Aurora: 7
Balmond: 8
Bane: 9
Belerick: 10
Bruno: 11
Chang`e: 12
Chou: 13
Claude: 14
Clint: 15
Cyclops: 16
Diggie: 17
Estes: 18
Eudora: 19
Fanny: 20
Franco: 21
Freya: 22
Gatot kaca: 23
Gord: 24
Grock: 25
Gusion: 26
Hanabi: 27
Harley: 28
Hayabusa: 29
Helcurt: 30
Hilda: 31
Hylos: 32
Irithel: 33
Jawhead: 34
Johnson: 35
Kagura: 36
Kaja: 37
Karina: 38
Karrie: 39
Lancelot: 40
Lapu-Lapu: 41
Layla: 42
Lesley: 43
Lolita: 44
Lunox: 45
Martis: 46
Minotaur: 47
Minsitthar: 48
Miya: 49
Moskov: 50
Natalia: 51
Odette: 52
Pharsa: 53
Rafaela: 54
Roger: 55
Ruby: 56
Saber: 57
Sun: 58
Thamuz: 59
Tigreal: 60
Uranus: 61
Valir: 62
Vexana: 63
Yi Sun-shin: 64
Zhask: 65
Zilong: 66

Mapping hero_role (Asli -> Encoded):
Assassin: 0
Fighter: 1
Fighter,Assassin: 2
Fighter,Support: 3
Mage: 4
Mage,Assassin: 5
Mage,Tank: 6
Marksman: 7
Marksman,Assassin: 8
Marksman,Fighter: 9
Support: 10
Tank: 11
Tank,Fighter: 12
Tank,

OSError: [Errno 9] Bad file descriptor

In [13]:
# Fitur
X = df[['hero_role', 'hero_specially',
        'hero_overview.hero_durability', 'hero_overview.hero_offence',
        'hero_overview.hero_ability', 'hero_overview.hero_difficulty']]
y = df['hero_name']

# Latih model KNN dengan seluruh dataset
knn = NearestNeighbors(n_neighbors=5)
knn.fit(X)

print("\nModel KNN berhasil dilatih.")

joblib.dump(knn, 'knn_model.pkl')


Model KNN berhasil dilatih.


OSError: [Errno 9] Bad file descriptor

In [14]:
# Muat label encoders dari file
label_encoders = joblib.load('label_encoders.pkl')

# Muat model KNN dari file
knn = joblib.load('knn_model.pkl')

# Input values (replace with your desired values)
hero_role_input = "Marksman"
hero_specially_input = "Crowd Control"
hero_durability_input = 0
hero_offence_input = 0
hero_ability_input = 0
hero_difficulty_input = 0

# Display labels for clarity
print("Role =", hero_role_input)
print("Specially =", hero_specially_input)
print("Durability =", hero_durability_input)
print("Offence =", hero_offence_input)
print("Ability =", hero_ability_input)
print("Difficulty =", hero_difficulty_input)

# Transformasi input kategorikal ke numerik
try:
    hero_role_encoded = label_encoders['hero_role'].transform([hero_role_input])[0]
except KeyError:
    print(f"Warning: Unseen label '{hero_role_input}' for hero_role. Using a placeholder value.")
    hero_role_encoded = -1  # Or any other suitable placeholder

try:
    hero_specially_encoded = label_encoders['hero_specially'].transform([hero_specially_input])[0]
except KeyError:
    print(f"Warning: Unseen label '{hero_specially_input}' for hero_specially. Using a placeholder value.")
    hero_specially_encoded = -1  # Or any other suitable placeholder

# Data input baru
new_data = [[hero_role_encoded, hero_specially_encoded, hero_durability_input,
             hero_offence_input, hero_ability_input, hero_difficulty_input]]

# Cari tetangga terdekat
distances, indices = knn.kneighbors(new_data)

print("\nTetangga terdekat:")
for i, index in enumerate(indices[0]):
    hero_name = label_encoders['hero_name'].inverse_transform([y.iloc[index]])[0]
    print(f"{i+1}. Hero: {hero_name}, Jarak: {distances[0][i]}")

Role = Marksman
Specially = Crowd Control
Durability = 0
Offence = 0
Ability = 0
Difficulty = 0

Tetangga terdekat:
1. Hero: Lancelot, Jarak: 5.0
2. Hero: Lapu-Lapu, Jarak: 6.4031242374328485
3. Hero: Minsitthar, Jarak: 7.0710678118654755
4. Hero: Gord, Jarak: 7.615773105863909
5. Hero: Kadita, Jarak: 8.06225774829855


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [7]:
# Hitung cosine similarity
cos_sim = cosine_similarity(new_data, X)

# Urutkan berdasarkan cosine similarity (descending)
sorted_indices = np.argsort(cos_sim[0])[::-1]

# Tampilkan hero berdasarkan cosine similarity
print("\nHero berdasarkan Cosine Similarity:")
for i in range(5):  # Tampilkan top 5 hero
    index = sorted_indices[i]
    hero_name = label_encoders['hero_name'].inverse_transform([y.iloc[index]])[0]
    print(f"{i+1}. Hero: {hero_name}, Cosine Similarity: {cos_sim[0][index]}")


Hero berdasarkan Cosine Similarity:
1. Hero: Harith, Cosine Similarity: 0.9658428372346681
2. Hero: Kimmy, Cosine Similarity: 0.9615239476408233
3. Hero: Dyrroth, Cosine Similarity: 0.961523947640823
4. Hero: Guinevere, Cosine Similarity: 0.9421026317177147
5. Hero: Baxia, Cosine Similarity: 0.9411201462819998


In [ ]:
# Fungsi untuk menghitung Average Precision
def average_precision(actual, predicted, k):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

# Fungsi untuk menghitung MAP@k
def mean_average_precision(actual, predicted, k):
    return np.mean([average_precision(a, p, k) for a, p in zip(actual, predicted)])

# Assuming 'y' contains the true hero names and 'indices' from knn.kneighbors
# contains the indices of the predicted neighbors:
actual_neighbors = [[y.iloc[i]] for i in range(len(y))] # Each actual neighbor is itself
predicted_neighbors = [[y.iloc[idx] for idx in row] for row in knn.kneighbors(X)[1]]

# Hitung MAP@5
map_score = mean_average_precision(actual_neighbors, predicted_neighbors, k=5)
print(f"MAP@5: {map_score:.2f}")

In [ ]:
# Fungsi untuk menghitung Top-K Accuracy
def top_k_accuracy(y_true, y_pred, k=5):
    """
    Menghitung Top-K Accuracy.

    Args:
        y_true: List/array dari label sebenarnya.
        y_pred: List/array dari label yang diprediksi (dalam bentuk indeks).
        k: Jumlah tetangga terdekat yang dipertimbangkan.

    Returns:
        Top-K Accuracy.
    """
    num_correct = 0
    for i in range(len(y_true)):
        # Periksa apakah label sebenarnya ada di antara k tetangga terdekat yang diprediksi
        if y_true[i] in [y.iloc[idx] for idx in y_pred[i][:k]]:
            num_correct += 1
    return num_correct / len(y_true)

# Hitung Top-K Accuracy
y_true = y.tolist()  # Ubah y menjadi list
# Dapatkan indeks tetangga terdekat (tanpa return_distances)
distances, indices = knn.kneighbors(X)
y_pred = indices
accuracy = top_k_accuracy(y_true, y_pred)

print(f"\nTop-5 Accuracy: {accuracy:.2f}")

In [ ]:
# 1. Definisikan Top-K Hero
k = 10  # Jumlah hero teratas yang direkomendasikan

# 2. Buat Prediksi
cos_sim = cosine_similarity(new_data, X)  # Hitung cosine similarity
predicted_labels = np.zeros(len(y))  # Inisialisasi label prediksi
top_k_predicted_indices = np.argsort(cos_sim[0])[::-1][:k]  # Indeks K hero teratas (prediksi)
predicted_labels[top_k_predicted_indices] = 1  # Label 1 untuk Top-K hero (prediksi)

# 3. Buat Label Aktual (Top-K Hero berdasarkan ground truth)
actual_labels = np.zeros(len(y))
ground_truth_indices = indices[0]  # Ganti dengan indeks hero yang seharusnya direkomendasikan
actual_labels[ground_truth_indices] = 1  # Label 1 untuk Top-K hero (ground truth)

# 4. Hitung dan Tampilkan Confusion Matrix
cm = confusion_matrix(actual_labels, predicted_labels)

print("Confusion Matrix:")
print(cm)

# Confusion matrix dari codingan kedua (Top-K Hero Berdasarkan Rekomendasi Ideal)
cm = [[80, 2],
      [2, 3]]

# Ekstrak nilai TP, TN, FP, dan FN dari confusion matrix
TP = cm[1][1]  # 3
TN = cm[0][0]  # 80
FP = cm[0][1]  # 2
FN = cm[1][0]  # 2

# Hitung akurasi
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Tampilkan akurasi
print(f"Akurasi Model: {accuracy:.2f}")

In [ ]:
# Mendapatkan prediksi untuk semua data
distances, indices = knn.kneighbors(X)

# Menyiapkan data untuk perhitungan MAP
y_true = np.zeros((len(y), len(np.unique(y))))
# One-hot encoding untuk y_true
for i, label in enumerate(y):
    y_true[i, label] = 1

y_pred = np.zeros_like(y_true)
for i, row in enumerate(indices):
    for j, idx in enumerate(row):
        y_pred[i, y[idx]] = 1  # Asumsikan y[idx] adalah label hero ke-idx

# Menghitung MAP
map_score = average_precision_score(y_true, y_pred, average='micro')

print(f"MAP: {map_score:.2f}")

In [ ]:
# Menghitung NDCG
ndcg_result = ndcg_score(y_true, y_pred) # change variable name to ndcg_result

print(f"NDCG: {ndcg_result:.2f}")

In [ ]:
# Menghitung Recall
recall = recall_score(y_true, y_pred, average='micro')

print(f"Recall: {recall:.2f}")